In [17]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import (
    StandardScaler,OneHotEncoder)
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

In [18]:
data=pd.read_csv('Train.csv')

In [19]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [20]:
n_transformers=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scalar', StandardScaler())
])

n_transformers

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scalar', StandardScaler())])

In [21]:
c_transformer = Pipeline(steps=[
        ('imputer',SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ])
c_transformer

Pipeline(steps=[('imputer',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [22]:
pd.set_option('display.max_columns',None)
import warnings

warnings.filterwarnings('ignore')

np.random.seed(42)
n = 1000
data1 = pd.DataFrame({
    'Age': np.random.normal(35 ,10,n),
    'Income': np.random.lognormal(10,1,n),
    'City': np.random.choice(['New York','London','Paris','Tokyo','Unknown']+[f'City_{i}' for i in range(50)],n),
    'Score': np.random.uniform(1,100,n),
    "Category_A" : np.random.choice(['A','B','C',np.nan],n, p =[0.3,0.3,0.3,0.1]),
    "Feature_x" : np.random.normal(0,1,n),
    "Feature_y" : np.random.normal(0,1,n),
    "Target" : np.random.randint(0,2,n)
})
data1.loc[1:10, 'Age']=150 
data1.to_csv("Synthetic Data")
data1.to_excel("Synthetic_Data.xlsx",index=False)


In [23]:
da=pd.read_csv('Synthetic Data')
da.head()

,Unnamed: 0,Age,Income,City,Score,Category_A,Feature_x,Feature_y,Target
0,0,39.967142,89264.168397,City_24,73.540614,C,0.204759,0.499096,1
1,1,150.000000,55527.500577,City_25,61.066441,C,0.765878,-1.692271,0
2,2,150.000000,23379.862874,City_32,67.614295,C,-1.615887,-0.170386,0
3,3,150.000000,11534.100901,City_5,9.033769,NaN,0.299648,-0.812824,0
4,4,150.000000,44277.118652,City_29,95.197580,A,0.932435,0.828059,0


In [24]:
d=pd.read_excel('Synthetic_Data.xlsx')
d

,Age,Income,City,Score,Category_A,Feature_x,Feature_y,Target
0,39.967142,89264.168397,City_24,73.540614,C,0.204759,0.499096,1
1,150.000000,55527.500577,City_25,61.066441,C,0.765878,-1.692271,0
2,150.000000,23379.862874,City_32,67.614295,C,-1.615887,-0.170386,0
3,150.000000,11534.100901,City_5,9.033769,NaN,0.299648,-0.812824,0
4,150.000000,44277.118652,City_29,95.197580,A,0.932435,0.828059,0
...,...,...,...,...,...,...,...,...
995,32.188997,64225.155190,City_42,71.757165,A,2.394362,0.069337,1
996,52.976865,21449.974612,City_28,30.041989,B,2.185095,0.862543,1
997,41.408429,9119.090469,City_34,53.021560,NaN,-0.686715,-1.694973,0
998,29.288210,18712.238164,City_5,70.085446,A,1.623885,0.820232,1


In [25]:
KNN_imputer = KNNImputer(n_neighbors= 5)
knn = KNN_imputer.fit_transform(data[['Weight_in_gms','Cost_of_the_Product','Discount_offered']])
print("KNN Imputation shape:",knn.shape)
type(knn)

KNN Imputation shape: (1000, 3)


numpy.ndarray

In [26]:
knn1 = pd.DataFrame(KNN_imputer.fit_transform(data[['Weight_in_gms','Cost_of_the_Product','Discount_offered']]),
                      columns = ['Weight_in_gms','Cost_of_the_Product','Discount_offered'])
knn1.describe()


,Weight_in_gms,Cost_of_the_Product,Discount_offered
count,1000.000000,1000.000000,1000.000000
mean,3501.413000,200.316000,31.987000
std,1407.924646,62.730178,18.506737
min,1005.000000,90.000000,1.000000
25%,2271.250000,146.000000,16.000000
50%,3544.000000,202.500000,32.000000
75%,4683.250000,254.000000,48.000000
max,5988.000000,309.000000,64.000000


In [27]:
ITER_imputer = IterativeImputer(max_iter=10,random_state=0)
iter = ITER_imputer.fit_transform(data[['Weight_in_gms','Cost_of_the_Product','Discount_offered']])
print("Imputation output shape:", iter.shape)
iter

Imputation output shape: (1000, 3)


array([[3838.,  306.,   45.],
       [2710.,  114.,   35.],
       [4152.,  215.,   44.],
       ...,
       [3584.,  187.,   64.],
       [1487.,  296.,   17.],
       [1304.,  207.,   17.]])

In [28]:
class Winsorizer(BaseEstimator, TransformerMixin):
    def __init__(self, low_quantile=0.01, up_quantile=0.99):
        self.low = low_quantile
        self.up = up_quantile
        self.limits_ = {}

    def fit(self, X, y=None):
        for col in X.columns:
            self.limits_[col] = (X[col].quantile(self.low), X[col].quantile(self.up))
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in X.columns:
            low, high = self.limits_[col]
            X_copy[col] = X_copy[col].clip(lower=low, upper=high)
        return X_copy

winsorizer = Winsorizer(up_quantile=0.95) # Cap at 95th percentile
df_capped = winsorizer.fit_transform(data[['Cost_of_the_Product']])
print(f"Max Cost Before: {data['Cost_of_the_Product'].max():.2f}, After: {df_capped['Cost_of_the_Product'].max():.2f}")

Max Cost Before: 309.00, After: 297.00


In [29]:
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

# --- GENERATE SYNTHETIC SHIPMENT DATA ---
# Creating a dataset that mimics the ShipmentSure problem (delays, costs, weights, etc.)
np.random.seed(42)
n = 1000
data2 = pd.DataFrame({
    'Warehouse_block': np.random.choice(['A', 'B', 'C', 'D', 'F'], n),
    'Mode_of_Shipment': np.random.choice(['Flight', 'Ship', 'Road', np.nan], n, p=[0.3, 0.3, 0.3, 0.1]), # Missing values example
    'Customer_care_calls': np.random.randint(2, 7, n),
    'Customer_rating': np.random.randint(1, 6, n),
    'Cost_of_the_Product': np.random.gamma(shape=2, scale=100, size=n), # Skewed distribution
    'Prior_purchases': np.random.randint(2, 10, n),
    'Product_importance': np.random.choice(['low', 'medium', 'high', 'critical'], n, p=[0.4, 0.4, 0.15, 0.05]),
    'Gender': np.random.choice(['M', 'F'], n),
    'Discount_offered': np.random.lognormal(2, 0.5, n).astype(int), # Highly skewed, potential outliers
    'Weight_in_gms': np.random.normal(4000, 1000, n),
    'Destination_City': np.random.choice(['New York', 'London', 'Paris', 'Tokyo', 'Unknown'] + [f'City_{i}' for i in range(50)], n), # High cardinality
    'Order_Hour': np.random.randint(0, 24, n),
    'Reached_on_Time': np.random.randint(0, 2, n)
})


data2.loc[0:20, 'Weight_in_gms'] = np.nan
data2.loc[900:910, 'Cost_of_the_Product'] = 5000 
X = data2.iloc[:,:-1]
y = data2.iloc[:,-1]
print("Synthetic Shipment Data Created.")
print(data2.head(3))
data2.to_csv('data2.csv',index=False)

Synthetic Shipment Data Created.
  Warehouse_block Mode_of_Shipment  Customer_care_calls  Customer_rating  \
0               D              nan                    4                2   
1               F             Ship                    2                2   
2               C             Road                    6                2   

   Cost_of_the_Product  Prior_purchases Product_importance Gender  \
0           311.610821                6                low      F   
1            49.650596                3             medium      M   
2           259.168778                3                low      M   

   Discount_offered  Weight_in_gms Destination_City  Order_Hour  \
0                 3            NaN           City_3          18   
1                 7            NaN          City_29          12   
2                 7            NaN          City_46          23   

   Reached_on_Time  
0                1  
1                0  
2                0  
